# Intro & Metadata

This notebook introduces the approach of the current repository, and explores the metadata files.

In [1]:
%config Completer.use_jedi = False

In [78]:
# Import local code module
import sys
import os
sys.path.append('..')
from conv import Paths, process_session
from conv.io import (get_files, load_configs, load_config, load_task_object,
                     save_config, save_task_object)
from conv.parser import parse_lines_log, parse_lines_sync
from conv.process import process_task

## Settings

In [79]:
# Define the examples folder
examples_folder = 'example_files/'

## Metadata Files

Metadata for the task is organized into a series of config files, each representing a facet of the study / task. 

In order to process an individual subject, the configs can be loaded together and session specific information can be added. 

This combined metadata file is then used to create the output data file. 

In [80]:
# Get a list of metadata files
files = get_files('../metadata', 'yaml')

In [81]:
# Check the list of configuration files
files

['device_info.yaml',
 'intervals_info.yaml',
 'position_info.yaml',
 'session_info.yaml',
 'sorting_info.yaml',
 'stimulus_info.yaml',
 'subject_info.yaml',
 'trial_info.yaml',
 'units_info.yaml']

In [82]:
# Load all the files together to create an overall metadata object
metadata = load_configs(files, '../metadata')

In [83]:
# Check the metadata object
metadata

{'device': {'device_name': 'Microwire Electrodes',
  'device_description': 'Behnke Fried/Micro Inner Wire Bundle',
  'device_manufacturer': 'Ad-Tech Medical',
  'bundle_description': 'Microwire Bundle'},
 'intervals': None,
 'position': {'player_position': 'Position of the player.',
  'head_direction': 'The head direction of the player, in degrees.',
  'speed': 'Movement speed of the player, computed from the position data.',
  'derived_position_measures': 'Derived measures related to position data.',
  'arena_center': 'Center of the arena.',
  'arena_x_range': 'Navigable range of the x-axis of the arena [min, max].',
  'arena_y_range': 'Navigable range of the y-axis of the arena [min, max].'},
 'session': {'identifier': 'XX',
  'session_id': 'XX',
  'session_start_time': '1111/11/11',
  'session_description': 'This file contains a recorded session of the XX task, a XX, from a human neurosurgical patient with microwires that record single-unit activity.',
  'experiment_name': 'XX',
  '

The above metadata file includes all the metadata fields

Where available, information is prefilled with default information for the current task. 

For fields not filled in (marked with `XX`), the metadata needs to be entered. 

In [84]:
# Save out the collected metadata to a subject-level file
save_config(metadata, 'example_metadata', folder=examples_folder)

In [85]:
# Reload the collected metadata file, and check an example attribute
metadata_new = load_config('example_metadata.yaml', folder=examples_folder)
metadata_new['session']['lab']

'Electrophysiology, Memory, and Navigation Laboratory'

## Logfile Processing

The task logfile, which is a structured txt file, needs parsing and organizing in order to create the new data files. 

In [86]:
# Define experiment label
experiment = 'movie_paradigm'

In [87]:
# Define subject information
subject = 'sub-561'
session = 'ses-1'

In [88]:
# Define project folder

# project_path = '/u/project/ifried/data/nwb_unpacked_recordings/'
project_path = '/Users/XinNiuAdmin/nwb_unpacked_recordings/'

In [89]:
# Define project paths object
paths = Paths(project_path, subject, experiment, session)

In [90]:
dir(paths)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_experiment',
 '_project_folders',
 '_recordings_name',
 '_session',
 '_session_folders',
 '_subject',
 '_subject_folders',
 'experiment',
 'get_files',
 'project',
 'recordings',
 'session',
 'session_name',
 'subject']

In [91]:
print(paths.behavior)
os.makedirs(paths.behavior, exist_ok=True)


/Users/XinNiuAdmin/nwb_unpacked_recordings/recordings/sub-561/movie_paradigm/session_ses-1/01_raw/behavior


### Process session all together

The logfile processing extracts required information from the logfile into a `Task` object.

In [44]:
# Process task information
task = process_session(paths, process=True)

		parsing logfile...


FileNotFoundError: [Errno 2] No such file or directory: '/u/project/ifried/data/nwb_unpacked_recordings/recordings/sub-561/movie_paradigm/session_ses-1/01_raw/behavior/logfile.txt'

In [ ]:
# Check the task object
task

### Process logfile in steps

The logfile processing includes two main steps:

- parse the logfile text, extracting required information
- process the collected information in the Task object

These processes can be run separately, which may be useful for exploring and checking for any issues, etc.

In [ ]:
# Parse the log file
task1 = parse_lines_log(paths.behavior)

In [ ]:
# Check task object - task information 
print(task1.trial['trial'][0:5])

In [ ]:
# Parse the sync pulses 
task1 = parse_lines_sync(paths.sync, task1)

This Task object can then be passed into a subsequent function to process the information.

In [ ]:
# Preprocess the task information
task1 = process_task(task1)

### Save & Reload task objects

In [ ]:
# Save out a task object
task_obj_name = 'example_task_obj'
save_task_object(task, task_obj_name, folder='example_files/')

In [ ]:
# Reload task object
new_task = load_task_object(task_obj_name, folder='example_files/')

In [ ]:
# Check reloaded task object
new_task

### Process Functions

This `Task` object can then be passed into subsequent function(s) to process the information.

These process functions include:
- `process_time_info`
- `process_task_info`
- `process_position_info`
- `process_location_info`
- `process_error_info`

Each of these functions can be updated independently if some task-related information needs to be updated. 

In [ ]:
# Import the process functions
from conv.process import *